<a href="https://colab.research.google.com/github/tomonari-masada/course2023-nlp/blob/main/08a_sentiment_analysis_with_LLM(Xwin_LM_13B_V0_1_GPTQ).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Appendix for LLMを使ってみる

* **ランタイムのタイプをGPUに設定しておくこと。**

In [ ]:
!pip install transformers datasets accelerate auto-gptq

**ここでランタイムを再起動する。**

### インポート

In [ ]:
import os
import numpy as np
import torch
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

np.random.seed(0)
torch.manual_seed(0)

## データセット
* WRIME ver.2
  * 主観と客観の感情分析データセット https://github.com/ids-cv/wrime


In [ ]:
dataset = load_dataset("shunk031/wrime", "ver2")
tags = ["train", "validation", "test"]

texts = {}
labels = {}
for tag in tags:
  texts[tag] = np.array(dataset[tag]["sentence"])
  labels[tag] = [item["sentiment"] for item in dataset[tag]["avg_readers"]]
  labels[tag] = np.array(labels[tag])

In [ ]:
texts_binary = {}
labels_binary = {}
for tag in tags:
  indices = labels[tag] != 0
  texts_binary[tag] = texts[tag][indices]
  labels_binary[tag] = labels[tag][indices]
  labels_binary[tag] = (labels_binary[tag] > 0) * 1

In [ ]:
label_to_text = ["悲しい", "嬉しい"]

## LLM


* 今回は、Xwin-LM-13B-V0.1を使う。
 * https://huggingface.co/Xwin-LM/Xwin-LM-13B-V0.1
* だが、Google Colab無料版では、この元のモデルは大きすぎて使えない・・・。
* そこで、量子化された下記のモデルを代わりに使う。
 * https://huggingface.co/TheBloke/Xwin-LM-13B-V0.1-GPTQ

### Xwin-LM-13B-V0.1-GPTQの取得
* モデルのダウンロードに少し時間がかかる。
* `AutoGPTQForCausalLM`クラスについては、以下を参照。
 * https://github.com/PanQiWei/AutoGPTQ/blob/main/auto_gptq/modeling/auto.py

* safetensorsについては、以下を参照。
 * https://huggingface.co/docs/diffusers/using-diffusers/using_safetensors

* `trust_remote_code`については、[ここ](https://huggingface.co/docs/transformers/model_doc/auto)に以下のような説明がある。

> Whether or not to allow for custom models defined on the Hub in their own modeling files. This option should only be set to True for repositories you trust and in which you have read the code, as it will execute code present on the Hub on your local machine.



In [ ]:
from auto_gptq import AutoGPTQForCausalLM

model_name = "TheBloke/Xwin-LM-13B-V0.1-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoGPTQForCausalLM.from_quantized(
    model_name,
    use_safetensors=True,
    inject_fused_attention=False,
    device="cuda:0",
    trust_remote_code=True,
    )
model.eval()

### In-context learning

In [ ]:
text = "Q:高い\nA:低い\n\nQ:大きい\nA:小さい\n\nQ:狭い\nA:広い\n\nQ:少ない\nA:多い\n\nQ:速い\nA:遅い\n\nQ:嬉しい\nA:"
print(text)

In [ ]:
token_ids = tokenizer.encode(text, return_tensors="pt")
output_ids = model.generate(
    input_ids=token_ids.to(model.device),
    max_new_tokens=10,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
)
output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):], skip_special_tokens=True)
print(output)

## 感情分析のプロンプト

### プロンプト作成用のヘルパ関数

In [ ]:
B_INST, E_INST = "", "答え："
B_SYS, E_SYS = "\n", "\n"
DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人のアシスタントです。"

def make_prompt(text):
  prompt = "「" + text + "」\nと言っている人の気持ちは、「嬉しい」と「悲しい」のうち、どちらですか。\n"
  return "{b_inst} {system}{prompt} {e_inst} ".format(
      b_inst=B_INST,
      system=f"{B_SYS}{DEFAULT_SYSTEM_PROMPT}{E_SYS}",
      prompt=prompt,
      e_inst=E_INST,
      ).strip()

## 感情分析

In [ ]:
for i in range(10):
  print(f'[{i+1}]' + '-'*80)
  prompt = make_prompt(texts_binary["train"][i])
  with torch.no_grad():
    token_ids = tokenizer.encode(prompt, return_tensors="pt")
    output_ids = model.generate(
        input_ids=token_ids.to(model.device),
        max_new_tokens=10,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
  output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):], skip_special_tokens=True)
  print(f"{prompt}\nprediction:{output}")
  print(f"ground truth:{label_to_text[labels_binary['train'][i]]}")
  print('-'*80)